In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.layers import LSTM, GRU
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D, Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
import pandas as pd
import tqdm
import random 
import string
from sklearn.metrics import confusion_matrix
import numpy as np
import keras
import pickle
import time
import re
import datetime
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
bucket_name='signature_classify'
file_location='sign_classify_f2/export/cl_model.hdf5'
tf_serving_model_out_location='gs://signature_classify/export/'

## Downloading the keras exported model
To download files from gcs, you can either use the python api or gsutil command. Here we will use gsutil.

In [3]:
!gsutil cp -r gs://signature_classify/job_dir/cl_model.hdf5 .



Updates are available for some Cloud SDK components.  To install them,
please run:
  $ gcloud components update

Copying gs://signature_classify/job_dir/cl_model.hdf5...
- [1 files][ 19.9 MiB/ 19.9 MiB]                                                
Operation completed over 1 objects/19.9 MiB.                                     


## Exporting the keras hdf5 model to Tensorflow SavedModel format and upload it to GCS

### Step 1 --> First define the model and load the weights

In [4]:
imag_shape = (100, 100, 3)
num_filters = 32            # No. of conv filters
max_pool_size = (2,2)       # shape of max_pool
conv_kernel_size = (3, 3)    # conv kernel shape
imag_shape = imag_shape
num_classes = 2
drop_prob = 0.5             # fraction to drop (0-1.0)

model = Sequential()
model.add(Conv2D(filters=num_filters, kernel_size=(conv_kernel_size), input_shape=imag_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=max_pool_size))

model.add(Conv2D(filters = num_filters*2, kernel_size=(conv_kernel_size), input_shape=imag_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=max_pool_size))

model.add(Conv2D(filters = num_filters*4, kernel_size=(conv_kernel_size), input_shape=imag_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=max_pool_size))

model.add(Flatten())
model.add(Dense(128, activation='relu'))  #Fully connected layer 

model.add(Dropout(drop_prob))

model.add(Dense(num_classes, activation='sigmoid'))

model.load_weights('cl_model.hdf5')

### Step 2 --> Export the model to SavedModel format using the below code

In [5]:
from tensorflow.keras import backend as K
from tensorflow.keras.models import model_from_config, Sequential

K.set_learning_phase(0) 
config = model.get_config()
weights = model.get_weights()

new_model = Sequential.from_config(config)
new_model.set_weights(weights)

from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model.signature_def_utils_impl import build_signature_def, predict_signature_def
from tensorflow.python.saved_model import tag_constants, signature_constants

%rm -rf signature_classify/export/version_1/
export_path = 'signature_classify/export/version_1/'

builder = saved_model_builder.SavedModelBuilder(export_path)

signature = predict_signature_def(inputs={'images': new_model.input},
                                  outputs={'scores': new_model.output})

with K.get_session() as sess:
    builder.add_meta_graph_and_variables(sess=sess,
                                         tags=[tag_constants.SERVING],
                                         signature_def_map={'serving_default': signature})
    builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: signature_classify/export/version_1/saved_model.pb


### Step 3 --> Verify the exported SavedModel using saved_model_cli api

In [6]:
!saved_model_cli show --dir signature_classify/export/version_1 --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['images'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 100, 100, 3)
        name: conv2d_input_1:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['scores'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 2)
        name: dense_1_1/Sigmoid:0
  Method name is: tensorflow/serving/predict


### Step 4 --> If you got an output like this, you can proceed to the next step. Now upload the folder where your SavedModel is written to gcs

In [7]:
!gsutil cp -r signature_classify gs://signature_classify

Copying file://signature_classify/.DS_Store [Content-Type=application/octet-stream]...
Copying file://signature_classify/export/.DS_Store [Content-Type=application/octet-stream]...
Copying file://signature_classify/export/version_1/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://signature_classify/export/version_1/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
| [4 files][ 13.3 MiB/ 13.3 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m -o ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://signature_classify/export/version_1/variables/variables.index [Content-Type=application/octet-stream]...
/ [5 files][ 13.3 MiB/ 13.3 MiB]                                                
Operation completed over 5 objects/13.3 Mi

## Deploying the  model on Google Cloud ML-Engine
### Step 1 --> Creating the model 

In [8]:
%env MODEL_NAME signature_classifier_model
%env MODEL_PATH gs://signature_classify/signature_classify/export/version_1

#CREATE MODEL
!gcloud ml-engine models create $MODEL_NAME 

env: MODEL_NAME=signature_classifier_model
env: MODEL_PATH=gs://signature_classify/signature_classify/export/version_1
Created ml engine model [projects/searce-sandbox/models/signature_classifier_model].


### Step 2 --> Creating version for the model you just created

In [60]:
!gcloud ml-engine versions create "version_3" --model $MODEL_NAME --origin $MODEL_PATH \
    --python-version 3.5 --runtime-version 1.10

Creating version (this might take a few minutes)......done.                    


### Step 3 --> Serving the model for predictions

In [58]:
from PIL import Image
import cv2

#Resize the original image.
img=Image.open('image.jpg')
img = img.resize((100, 100), Image.ANTIALIAS)
img.save('new_image.jpg')

img=cv2.imread('new_image.jpg')
input_dict={'images': img.tolist()}

import json
with open('test_data.json', 'w') as outfile:
    json.dump(input_dict, outfile)

(100, 100, 3)


In [61]:
!gcloud ml-engine predict --model $MODEL_NAME --version version_3 --json-instances test_data.json

SCORES
[1.0, 0.0]
